# Geometrical relaxations from pretrained m3gnet model

In [1]:
import ML_library as MLL
import ase 
import matgl
import warnings

from ase.io                      import read, write
from ase.io.vasp                 import write_vasp_xdatcar
from pymatgen.io.vasp.inputs     import Poscar
from os                          import path
from __future__                  import annotations
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from pymatgen.core               import Lattice, Structure
from pymatgen.io.ase             import AseAtomsAdaptor
from matgl.ext.ase               import M3GNetCalculator, MolecularDynamics, Relaxer
from pymatgen.io.ase             import AseAtomsAdaptor

# To suppress warnings for clearer output
warnings.simplefilter('ignore')

In [2]:
# Define paths to pretrained model and structure to be relaxed

# Materials Project pretrained model as default
model_load_path = 'finetuned_model'
model_load_path = 'M3GNet-MP-2021.2.8-PES' if model_load_path is None else model_load_path
model_load_path = 'M3GNet-MP-2021.2.8-PES'

path_to_POSCAR  = 'examples'

# Structure Relaxation

To perform structure relaxation, we use the Relaxer class.

In [3]:
# Load the structure to be relaxed
atoms_ini = Structure.from_file(f'{path_to_POSCAR}/POSCAR')

In [4]:
# Clear caché of previous model
#matgl.clear_cache()

# Load the default pre-trained model
pot = matgl.load_model(model_load_path)
relaxer = Relaxer(potential=pot)

In [5]:
# Relax the structure
relax_atoms_ini = relaxer.relax(atoms_ini, verbose=True)
atoms = relax_atoms_ini['final_structure']

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 23:34:04      -60.460709*      26.5589
FIRE:    1 23:34:04      -55.440254*      67.7939
FIRE:    2 23:34:04      -60.466652*      26.5406
FIRE:    3 23:34:04      -60.967625*       9.2130
FIRE:    4 23:34:04      -60.914852*      10.6447
FIRE:    5 23:34:05      -60.943428*       9.4187
FIRE:    6 23:34:05      -60.987305*       7.0617
FIRE:    7 23:34:05      -61.025490*       3.7695
FIRE:    8 23:34:06      -61.038826*       1.5587
FIRE:    9 23:34:06      -61.038937*       1.5364
FIRE:   10 23:34:07      -61.039150*       1.4924
FIRE:   11 23:34:08      -61.039459*       1.4284
FIRE:   12 23:34:08      -61.039860*       1.3466
FIRE:   13 23:34:09      -61.040340*       1.2502
FIRE:   14 23:34:10      -61.040894*       1.1432
FIRE:   15 23:34:10      -61.041504*       1.0301
FIRE:   16 23:34:11      -61.042255*       0.9039
FIRE:   17 23:34:11      -61.043167*       0.7701
FI

In [6]:
# Save the relaxed structure as a POSCAR file
poscar_relaxed = Poscar(atoms)
poscar_relaxed.write_file(f'{path_to_POSCAR}/CONTCAR')

In [7]:
# Or as implemented
poscar_relaxed = MLL.structural_relaxation(path_to_POSCAR, model_load_path)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 23:34:29      -60.460709*      26.5589
FIRE:    1 23:34:29      -55.440250*      67.7939
FIRE:    2 23:34:29      -60.466656*      26.5406
FIRE:    3 23:34:29      -60.967625*       9.2131
FIRE:    4 23:34:29      -60.914856*      10.6447
FIRE:    5 23:34:29      -60.943428*       9.4187
FIRE:    6 23:34:30      -60.987305*       7.0617
FIRE:    7 23:34:30      -61.025494*       3.7695
FIRE:    8 23:34:30      -61.038830*       1.5587
FIRE:    9 23:34:30      -61.038937*       1.5364
FIRE:   10 23:34:30      -61.039150*       1.4925
FIRE:   11 23:34:30      -61.039459*       1.4284
FIRE:   12 23:34:30      -61.039860*       1.3466
FIRE:   13 23:34:31      -61.040337*       1.2502
FIRE:   14 23:34:31      -61.040890*       1.1431
FIRE:   15 23:34:31      -61.041504*       1.0301
FIRE:   16 23:34:31      -61.042255*       0.9039
FIRE:   17 23:34:31      -61.043167*       0.7701
FI

# Single-shot energy calculation

Perform a single-shot calculation for final structure using M3GNetCalculator.

In [8]:
# Define the M3GNet calculator
calc = M3GNetCalculator(pot)

# Load atoms adapter and adapt structure
ase_adaptor = AseAtomsAdaptor()
adapted_atoms = ase_adaptor.get_atoms(atoms)

# Calculate potential energy
adapted_atoms.set_calculator(calc)
print(f'The calculated potential energy is {float(adapted_atoms.get_potential_energy()):.3f} eV/supercell')

The calculated potential energy is -61.342 eV/supercell


In [9]:
# Or as implemented
ssc_energy = MLL.single_shot_energy_calculations(f'{path_to_POSCAR}/CONTCAR', model_load_path)
print(f'The calculated potential energy is {ssc_energy:.3f} eV/supercell')

The calculated potential energy is -61.342 eV/supercell


In [10]:
# Try on 3x3x3 supercell
ssc_energy = MLL.single_shot_energy_calculations(f'{path_to_POSCAR}/333_supercell/POSCAR', model_load_path)
print(f'The calculated potential energy is {ssc_energy:.3f} eV/supercell')

The calculated potential energy is -1656.253 eV/supercell
